In [2]:
import sqlalchemy

In [3]:
sqlalchemy.__version__

'1.2.7'

In [4]:
from sqlalchemy import create_engine

In [140]:
engine=create_engine("sqlite:///alchemytest.db",echo=True)

In [141]:
print(engine)

Engine(sqlite:///alchemytest.db)


In [142]:
from sqlalchemy import MetaData

In [143]:
metadata = MetaData()

In [144]:
print(metadata)

MetaData(bind=None)


In [145]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey

In [146]:
users=Table('users', metadata,
            Column('id',Integer, primary_key=True),
            Column('name', String),
            Column('fullname',String),
)

address = Table('address',metadata,
            Column('id',Integer, primary_key=True),
            Column('user_id', None, ForeignKey('users.id')),
            Column('email_address',String, nullable=False),
)

metadata.create_all(engine)

2018-07-10 16:23:15,812 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:23:15,813 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:15,815 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:23:15,815 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:15,816 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-10 16:23:15,817 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:15,818 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("address")
2018-07-10 16:23:15,823 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:15,824 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2018-07-10 16:23:15,826 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:15,829 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:23:15,830 INFO sqlalchemy.engine.bas

In [147]:
for i in metadata.tables:
    print(i)

users
address


In [148]:
insert=users.insert()
print(insert)

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [149]:
insert=users.insert().values(name='kim', fullname='Kim Dong Han')
print(insert)

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


In [150]:
insert.compile().params

{'name': 'kim', 'fullname': 'Kim Dong Han'}

In [151]:
insert.bind=engine#엔진에 바인딩
conn=engine.connect()

In [152]:
result=conn.execute(insert)

2018-07-10 16:23:16,048 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 16:23:16,049 INFO sqlalchemy.engine.base.Engine ('kim', 'Kim Dong Han')
2018-07-10 16:23:16,052 INFO sqlalchemy.engine.base.Engine COMMIT


In [153]:
result.inserted_primary_key

[1]

In [154]:
conn.execute(insert, name="lee", fullname="unknown lee")

2018-07-10 16:23:16,135 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2018-07-10 16:23:16,136 INFO sqlalchemy.engine.base.Engine ('lee', 'unknown lee')
2018-07-10 16:23:16,139 INFO sqlalchemy.engine.base.Engine COMMIT


In [155]:
result.inserted_primary_key

[1]

In [156]:
conn.execute(address.insert(),[
    {'user_id':1,"email_address":"houhou@naver.com"},
    {'user_id':2,"email_address":"hou@naver.com"}
])

2018-07-10 16:23:16,196 INFO sqlalchemy.engine.base.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?)
2018-07-10 16:23:16,197 INFO sqlalchemy.engine.base.Engine ((1, 'houhou@naver.com'), (2, 'hou@naver.com'))
2018-07-10 16:23:16,199 INFO sqlalchemy.engine.base.Engine COMMIT


In [157]:
from sqlalchemy.sql import select

In [158]:
query=select([users])
result=conn.execute(query)

for i in result:
    print(i)

2018-07-10 16:23:16,310 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 16:23:16,312 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Kim Dong Han')
(2, 'lee', 'unknown lee')


In [159]:
result=conn.execute(select([users.c.name, users.c.fullname]))

for i in result:
    print(i)

2018-07-10 16:23:16,348 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2018-07-10 16:23:16,349 INFO sqlalchemy.engine.base.Engine ()
('kim', 'Kim Dong Han')
('lee', 'unknown lee')


In [160]:
result=conn.execute(query)

row =result.fetchone()
print("id-",row["id"], ", name -", row["name"], ",fullname-",row["fullname"])

row =result.fetchone()
print("id-",row[0], ", name -", row[1], ",fullname-",row[2])


result = conn.execute(query)
rows = result.fetchall()

for i in rows:
    print("id-",row[0], ", name -", row[1], ",fullname-",row[2])
    
result.close()


2018-07-10 16:23:16,393 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 16:23:16,394 INFO sqlalchemy.engine.base.Engine ()
id- 1 , name - kim ,fullname- Kim Dong Han
id- 2 , name - lee ,fullname- unknown lee
2018-07-10 16:23:16,397 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2018-07-10 16:23:16,398 INFO sqlalchemy.engine.base.Engine ()
id- 2 , name - lee ,fullname- unknown lee
id- 2 , name - lee ,fullname- unknown lee


In [161]:
print((users.c.id == address.c.user_id) | (users.c.id ==1) & (address.c.email_address.like("a%")))

users.id = address.user_id OR users.id = :id_1 AND address.email_address LIKE :email_address_1


In [162]:
result= conn.execute(select([users]).where(users.c.id==1))

2018-07-10 16:23:16,450 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2018-07-10 16:23:16,451 INFO sqlalchemy.engine.base.Engine (1,)


In [163]:
for row in result:
    print(row)

(1, 'kim', 'Kim Dong Han')


In [164]:
result= conn.execute(select([users, address]).where(users.c.id==address.c.user_id))

2018-07-10 16:23:16,505 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, address.id, address.user_id, address.email_address 
FROM users, address 
WHERE users.id = address.user_id
2018-07-10 16:23:16,506 INFO sqlalchemy.engine.base.Engine ()


In [165]:
result=conn.execute(select([users]).where(users.c.name.like("k%")))

2018-07-10 16:23:16,532 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name LIKE ?
2018-07-10 16:23:16,533 INFO sqlalchemy.engine.base.Engine ('k%',)


In [166]:
for row in result:
    print(row)

(1, 'kim', 'Kim Dong Han')


In [167]:
result = conn.execute(select([users, address]).where(users.c.id==address.c.user_id).where(users.c.name.like("k%")))

2018-07-10 16:23:16,641 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, address.id, address.user_id, address.email_address 
FROM users, address 
WHERE users.id = address.user_id AND users.name LIKE ?
2018-07-10 16:23:16,643 INFO sqlalchemy.engine.base.Engine ('k%',)


In [168]:
for row in result:
    print(row)

(1, 'kim', 'Kim Dong Han', 1, 1, 'houhou@naver.com')


In [169]:
from sqlalchemy import join

In [170]:
print(users.join(address, users.c.id==address.c.user_id))

users JOIN address ON users.id = address.user_id


#### inner join

In [171]:
result=conn.execute(select([users.c.name, address.c.email_address]).select_from(users.join(address)))

2018-07-10 16:23:16,758 INFO sqlalchemy.engine.base.Engine SELECT users.name, address.email_address 
FROM users JOIN address ON users.id = address.user_id
2018-07-10 16:23:16,759 INFO sqlalchemy.engine.base.Engine ()


In [172]:
for i in result:
    print(i)

('kim', 'houhou@naver.com')
('lee', 'hou@naver.com')


In [173]:
artist=Table("Artist",metadata,
            Column("id", Integer, primary_key=True),
            Column("name", String, nullable=False),
            extened_existing=True)

album=Table("Album",metadata,
            Column("id", Integer, primary_key=True),
            Column("title",String, nullable=False),
            Column("artist_id",Integer, ForeignKey("Artist.id")),
            extened_existing=True)

genre=Table("Genre",metadata,
            Column("id", Integer, primary_key=True),
            Column("name",String, nullable=False),
            extened_existing=True)

track=Table("Track",metadata,
            Column("id", Integer, primary_key=True),
            Column("title", String, nullable=False),
            Column("length", Integer, nullable=False),
            Column("rating", Integer, nullable=False),
            Column("count", Integer, nullable=False),
            Column("album_id",Integer, ForeignKey("Album.id")),
            Column("genre_id",Integer, ForeignKey("Genre.id")),
            extened_existing=True)

metadata.create_all(engine)

2018-07-10 16:23:16,900 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2018-07-10 16:23:16,901 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:16,905 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("address")
2018-07-10 16:23:16,905 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:16,907 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2018-07-10 16:23:16,907 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:16,909 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-10 16:23:16,909 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:16,911 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2018-07-10 16:23:16,911 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:16,912 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2018-07-10 16:23:16,913 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:23:16,914 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 
	n

C:\Users\user\Anaconda3\lib\site-packages\sqlalchemy\sql\base.py:291: SAWarning: Can't validate argument 'extened_existing'; can't locate any SQLAlchemy dialect named 'extened'
  (k, dialect_name))


In [174]:
for i in metadata.tables:
    print(i)

users
address
Artist
Album
Genre
Track


In [175]:
for i in engine.table_names():
    print(i)

2018-07-10 16:23:16,961 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-10 16:23:16,962 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track
address
users


In [176]:
conn.execute(artist.insert(), [
    {"name":"Led zepplin"},
    {"name":"AC/DC"}
])

conn.execute(album.insert(), [
    {"title":"IV", "artist_id":1},
    {"title":"WHo Made Who", "artist_id":2}
])

conn.execute(genre.insert(), [
    {"name":"Rock"},
    {"name":"Metal"}
])

conn.execute(track.insert(), [
    {"title":"Black Dog", "rating":5, "length":297, "count":0, "album_id":1, "genre_id":1},
    {"title":"StairWay", "rating":5, "length":482, "count":0, "album_id":1, "genre_id":1},
    {"title":"About to Rock", "rating":5, "length":313, "count":0, "album_id":2, "genre_id":2},
    {"title":"Who Made Who", "rating":5, "length":297, "count":0, "album_id":2, "genre_id":2},
    
])

2018-07-10 16:23:17,011 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2018-07-10 16:23:17,012 INFO sqlalchemy.engine.base.Engine (('Led zepplin',), ('AC/DC',))
2018-07-10 16:23:17,014 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:23:17,017 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2018-07-10 16:23:17,018 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('WHo Made Who', 2))
2018-07-10 16:23:17,020 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:23:17,022 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2018-07-10 16:23:17,023 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2018-07-10 16:23:17,025 INFO sqlalchemy.engine.base.Engine COMMIT
2018-07-10 16:23:17,028 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2018-07-10 16:23:17,029 INFO sqlalchemy.engine.base.Engine (('Black Dog', 

In [177]:
result=conn.execute(artist.select())
for i in result:
    print(i)

2018-07-10 16:23:17,059 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id, "Artist".name 
FROM "Artist"
2018-07-10 16:23:17,061 INFO sqlalchemy.engine.base.Engine ()
(1, 'Led zepplin')
(2, 'AC/DC')


In [178]:
result=conn.execute(track.select())
for i in result:
    print(i)

2018-07-10 16:23:17,097 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track"
2018-07-10 16:23:17,098 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'StairWay', 482, 5, 0, 1, 1)
(3, 'About to Rock', 313, 5, 0, 2, 2)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [179]:
from sqlalchemy import and_, or_
result=conn.execute(select([track])
                           .where(
                               and_(track.c.album_id==1,
                                   or_(track.c.genre_id==1,
                                       track.c.genre_id==2,)
                                   )
                           )
                    )
                    
for i in result:
    print(i)

2018-07-10 16:23:17,141 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)
2018-07-10 16:23:17,143 INFO sqlalchemy.engine.base.Engine (1, 1, 2)
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'StairWay', 482, 5, 0, 1, 1)


In [180]:
from sqlalchemy import update

In [181]:
conn.execute(track.update().values(genre_id=1).where(track.c.genre_id==3))

2018-07-10 16:23:17,204 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".genre_id = ?
2018-07-10 16:23:17,205 INFO sqlalchemy.engine.base.Engine (1, 3)
2018-07-10 16:23:17,207 INFO sqlalchemy.engine.base.Engine COMMIT


In [182]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [183]:
result=conn.execute(track.select().select_from(track.join(album)))
for i in result:
    print(i)

2018-07-10 16:23:17,313 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2018-07-10 16:23:17,315 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'StairWay', 482, 5, 0, 1, 1)
(3, 'About to Rock', 313, 5, 0, 2, 2)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [184]:
result=conn.execute(track.select().select_from(track.join(album)))
for i in result:
    print(i)

2018-07-10 16:23:17,349 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2018-07-10 16:23:17,350 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'StairWay', 482, 5, 0, 1, 1)
(3, 'About to Rock', 313, 5, 0, 2, 2)
(4, 'Who Made Who', 297, 5, 0, 2, 2)


In [185]:
print(track.join(album))
print(track.join(album).join(genre))
print(track.join(album).join(artist))
print(track.join(album).join(genre).join(artist))

result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name]).select_from(track.join(album).join(genre).join(artist)))

for i in result.fetchall():
    print(i)

"Track" JOIN "Album" ON "Album".id = "Track".album_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Artist" ON "Artist".id = "Album".artist_id
"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2018-07-10 16:23:17,399 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2018-07-10 16:23:17,401 INFO sqlalchemy.engine.base.Engine ()
('Black Dog', 'IV', 'Rock', 'Led zepplin')
('StairWay', 'IV', 'Rock', 'Led zepplin')
('About to Rock', 'WHo Made Who', 'Metal', 'AC/DC')
('Who Made Who', 'WHo Made Who', 'Metal', 'AC/DC')


In [186]:
result=conn.execute(track.select().select_from(track.join(album).join(genre).join(artist)).where(and_(genre.c.id==1,artist.c.id==1,)))

for i in result.fetchall():
    print(i)

2018-07-10 16:23:17,447 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id 
WHERE "Genre".id = ? AND "Artist".id = ?
2018-07-10 16:23:17,448 INFO sqlalchemy.engine.base.Engine (1, 1)
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'StairWay', 482, 5, 0, 1, 1)


In [187]:
result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name]).select_from(track.join(album).join(genre).join(artist))
                     .where(
                     and_(
                     genre.c.id==1,
                     artist.c.name.like('%pp%')
                     )
                     ))

for i in result.fetchall():
    print(i)

2018-07-10 16:23:17,499 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id 
WHERE "Genre".id = ? AND "Artist".name LIKE ?
2018-07-10 16:23:17,500 INFO sqlalchemy.engine.base.Engine (1, '%pp%')
('Black Dog', 'IV', 'Rock', 'Led zepplin')
('StairWay', 'IV', 'Rock', 'Led zepplin')


In [6]:
from sqlalchemy import create_engine, MetaData

engine=create_engine("sqlite:///alchemytest.db",echo=True)
conn=engine.connect()

metadata=MetaData(bind=engine, reflect=True)
metadata.reflect(bind=engine)

for i in metadata.tables:
    print(i)

2018-07-10 16:32:30,076 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:32:30,079 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:32:30,082 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:32:30,083 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:32:30,086 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-10 16:32:30,087 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:32:30,090 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-10 16:32:30,092 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:32:30,094 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-07-10 16:32:30,095 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:32:30,097 INFO sqlalchemy.engine.base.

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  


In [7]:
for i in metadata.tables:
    print(i)

Album
Artist
Genre
Track


In [9]:
album=metadata.tables["Album"]

In [10]:
track

NameError: name 'track' is not defined

In [11]:
album

Table('Album', MetaData(bind=Engine(sqlite:///alchemytest.db)), Column('id', INTEGER(), table=<Album>, primary_key=True, nullable=False), Column('title', VARCHAR(), table=<Album>, nullable=False), Column('artist_id', INTEGER(), ForeignKey('Artist.id'), table=<Album>), schema=None)

In [13]:
result=conn.execute(album.select())

2018-07-10 16:34:48,071 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"
2018-07-10 16:34:48,072 INFO sqlalchemy.engine.base.Engine ()


In [15]:
for i in result:
    print(i)

(1, 'IV', 1)
(2, 'WHo Made Who', 2)
